In [1]:
import os
import lpips
import cv2
import torch
import numpy as np
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Task 1

In [2]:
# Get the list of image files in the folder
image_files = ["256x256.png", "512x512.png", "1000x1000.png", "2000x2000.png"]

# Loop over each image and convert to grayscale
for image_file in image_files:
    # Open the image
    img = Image.open(os.path.join('assets', image_file))

    # Convert the image to grayscale
    grayscale_img = img.convert("L")  # 'L' mode stands for grayscale

    # Save the grayscale image to the output folder
    grayscale_img.save(os.path.join('assets', f"grayscale_{image_file}"))

# Task 2.1

In [3]:
grayscaled_images = ["grayscale_256x256.png", "grayscale_512x512.png", "grayscale_1000x1000.png", "grayscale_2000x2000.png"]
for image_file in grayscaled_images:
    # Open the PNG image
    img = Image.open(f"assets/{image_file}")

    # Convert to JPEG format and save with compression
    img = img.convert("RGB")
    img.save(f"assets/{image_file}_compressed_q85.jpg", "JPEG", quality=85)  # 'quality' ranges from 1 (worst) to 95 (best)
    img.save(f"assets/{image_file}_compressed_q5.jpg", "JPEG", quality=5)  # 'quality' ranges from 1 (worst) to 95 (best)

In [4]:
for image_file in grayscaled_images:
    # Open the PNG image
    img = Image.open(f"assets/{image_file}")
    img = img.convert("RGB")  # Convert to RGB format if necessary

    # Convert image to numpy array
    img_array = np.array(img)

    for stddev in [25, 85]:
        # Define Gaussian noise parameters
        mean = 0

        # Generate Gaussian noise
        gaussian_noise = np.random.normal(mean, stddev, img_array.shape)

        # Add the Gaussian noise to the image
        noisy_img_array = img_array + gaussian_noise

        # Ensure pixel values are within [0, 255] range
        noisy_img_array = np.clip(noisy_img_array, 0, 255).astype(np.uint8)

        # Convert back to a PIL image
        noisy_img = Image.fromarray(noisy_img_array)

        # Save the noisy image
        noisy_img.save(f"assets/gaussian{stddev}_{image_file}")


# Task 3

In [14]:
# Function to compute MSE between two images
def mse(imageA, imageB):
    # Compute the mean squared error
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    return err

# Load images (grayscale images)
for image_file in grayscaled_images:
    original = cv2.imread(f"assets/{image_file}", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q85 = cv2.imread(f"assets/{image_file}_compressed_q85.jpg", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q5 = cv2.imread(f"assets/{image_file}_compressed_q5.jpg", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_85 = cv2.imread(f"assets/gaussian85_{image_file}", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_25 = cv2.imread(f"assets/gaussian25_{image_file}", cv2.IMREAD_GRAYSCALE)

    # Calculate MSE between the original and distorted images
    mse_jpeg_85 = mse(original, jpeg_compressed_q85)
    mse_jpeg_5 = mse(original, jpeg_compressed_q5)
    mse_noise_85 = mse(original, gaussian_noise_85)
    mse_noise_25 = mse(original, gaussian_noise_25)

    print(f"MSE for JPEG compressed q85 image: {mse_jpeg_85}")
    print(f"MSE for JPEG compressed q5 image: {mse_jpeg_5}")
    print(f"MSE for image with Gaussian noise 85: {mse_noise_85}")
    print(f"MSE for image with Gaussian noise 25: {mse_noise_25}\n")

MSE for JPEG compressed q85 image: 21.508682250976562
MSE for JPEG compressed q5 image: 239.91409301757812
MSE for image with Gaussian noise 85: 2205.6570587158203
MSE for image with Gaussian noise 25: 262.55091857910156

MSE for JPEG compressed q85 image: 18.49268341064453
MSE for JPEG compressed q5 image: 293.5249938964844
MSE for image with Gaussian noise 85: 2229.9584617614746
MSE for image with Gaussian noise 25: 263.73778915405273

MSE for JPEG compressed q85 image: 2.642072
MSE for JPEG compressed q5 image: 74.442835
MSE for image with Gaussian noise 85: 2269.365355
MSE for image with Gaussian noise 25: 268.080716

MSE for JPEG compressed q85 image: 1.05619
MSE for JPEG compressed q5 image: 66.402881
MSE for image with Gaussian noise 85: 2133.43445675
MSE for image with Gaussian noise 25: 184.2664925



libpng warning: iCCP: profile 'ICC Profile': 'RGB ': RGB color space not permitted on grayscale PNG


In [16]:
# Function to compute PSNR
def psnr(imageA, imageB):
    mse_value = mse(imageA, imageB)
    if mse_value == 0:
        return float('inf')  # PSNR is infinite if MSE is zero (identical images)
    max_pixel_value = 255.0
    return 10 * np.log10((max_pixel_value ** 2) / mse_value)

for image_file in grayscaled_images:
    original = cv2.imread(f"assets/{image_file}", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q85 = cv2.imread(f"assets/{image_file}_compressed_q85.jpg", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q5 = cv2.imread(f"assets/{image_file}_compressed_q5.jpg", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_85 = cv2.imread(f"assets/gaussian85_{image_file}", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_25 = cv2.imread(f"assets/gaussian25_{image_file}", cv2.IMREAD_GRAYSCALE)
    
    psnr_jpeg_85 = psnr(original, jpeg_compressed_q85)
    psnr_jpeg_5 = psnr(original, jpeg_compressed_q5)
    psnr_noise_85 = psnr(original, gaussian_noise_85)
    psnr_noise_25 = psnr(original, gaussian_noise_25)

    print(f"PSNR for JPEG compressed q85 image: {psnr_jpeg_85} dB")
    print(f"PSNR for JPEG compressed q5 image: {psnr_jpeg_85} dB")
    print(f"PSNR for image with Gaussian noise 85: {psnr_noise_85} dB")
    print(f"PSNR for image with Gaussian noise 25: {psnr_noise_25} dB\n")

PSNR for JPEG compressed q85 image: 34.80466557112713 dB
PSNR for JPEG compressed q5 image: 34.80466557112713 dB
PSNR for image with Gaussian noise 85: 14.695423727480229 dB
PSNR for image with Gaussian noise 25: 23.938668187889704 dB

PSNR for JPEG compressed q85 image: 35.46080426134991 dB
PSNR for JPEG compressed q5 image: 35.46080426134991 dB
PSNR for image with Gaussian noise 85: 14.647835875039629 dB
PSNR for image with Gaussian noise 25: 23.919079995793254 dB

PSNR for JPEG compressed q85 image: 43.91135712321942 dB
PSNR for JPEG compressed q5 image: 43.91135712321942 dB
PSNR for image with Gaussian noise 85: 14.571759403963151 dB
PSNR for image with Gaussian noise 25: 23.848147861092052 dB

PSNR for JPEG compressed q85 image: 47.893383095932776 dB
PSNR for JPEG compressed q5 image: 47.893383095932776 dB
PSNR for image with Gaussian noise 85: 14.840010558375889 dB
PSNR for image with Gaussian noise 25: 25.476339917116153 dB



libpng warning: iCCP: profile 'ICC Profile': 'RGB ': RGB color space not permitted on grayscale PNG


In [18]:
for image_file in grayscaled_images:
    original = cv2.imread(f"assets/{image_file}", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q85 = cv2.imread(f"assets/{image_file}_compressed_q85.jpg", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q5 = cv2.imread(f"assets/{image_file}_compressed_q5.jpg", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_85 = cv2.imread(f"assets/gaussian85_{image_file}", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_25 = cv2.imread(f"assets/gaussian25_{image_file}", cv2.IMREAD_GRAYSCALE)

    ssim_jpeg_85 = ssim(original, jpeg_compressed_q85)
    ssim_jpeg_5 = ssim(original, jpeg_compressed_q5)
    ssim_noise_85 = ssim(original, gaussian_noise_85)
    ssim_noise_25 = ssim(original, gaussian_noise_25)

    print(f"SSIM for JPEG compressed q85 image: {ssim_jpeg_85}")
    print(f"SSIM for JPEG compressed q5 image: {ssim_jpeg_5}")
    print(f"SSIM for image with Gaussian noise 85: {ssim_noise_85}")
    print(f"SSIM for image with Gaussian noise 25: {ssim_noise_25}\n")

SSIM for JPEG compressed q85 image: 0.9416431808231807
SSIM for JPEG compressed q5 image: 0.5362863452715882
SSIM for image with Gaussian noise 85: 0.1692192179584619
SSIM for image with Gaussian noise 25: 0.5647442555127854

SSIM for JPEG compressed q85 image: 0.9618688799688425
SSIM for JPEG compressed q5 image: 0.5797062614359662
SSIM for image with Gaussian noise 85: 0.20268074488668306
SSIM for image with Gaussian noise 25: 0.6142521132218415

SSIM for JPEG compressed q85 image: 0.9872683859722619
SSIM for JPEG compressed q5 image: 0.8379614795159436
SSIM for image with Gaussian noise 85: 0.06173434515785253
SSIM for image with Gaussian noise 25: 0.27799044993851924



libpng warning: iCCP: profile 'ICC Profile': 'RGB ': RGB color space not permitted on grayscale PNG


SSIM for JPEG compressed q85 image: 0.9939585222470035
SSIM for JPEG compressed q5 image: 0.35898882122947356
SSIM for image with Gaussian noise 85: 0.027127364587235626
SSIM for image with Gaussian noise 25: 0.1579069814721672



In [20]:
for image_file in grayscaled_images:
    original = cv2.imread(f"assets/{image_file}", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q85 = cv2.imread(f"assets/{image_file}_compressed_q85.jpg", cv2.IMREAD_GRAYSCALE)
    jpeg_compressed_q5 = cv2.imread(f"assets/{image_file}_compressed_q5.jpg", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_85 = cv2.imread(f"assets/gaussian85_{image_file}", cv2.IMREAD_GRAYSCALE)
    gaussian_noise_25 = cv2.imread(f"assets/gaussian25_{image_file}", cv2.IMREAD_GRAYSCALE)

    # Convert grayscale to RGB by repeating the channel and normalize
    original_rgb = cv2.cvtColor(original, cv2.COLOR_GRAY2RGB) / 255.0
    jpeg_compressed_85_rgb = cv2.cvtColor(jpeg_compressed_q85, cv2.COLOR_GRAY2RGB) / 255.0
    jpeg_compressed_5_rgb = cv2.cvtColor(jpeg_compressed_q5, cv2.COLOR_GRAY2RGB) / 255.0
    gaussian_noise_85_rgb = cv2.cvtColor(gaussian_noise_85, cv2.COLOR_GRAY2RGB) / 255.0
    gaussian_noise_25_rgb = cv2.cvtColor(gaussian_noise_85, cv2.COLOR_GRAY2RGB) / 255.0

    # Convert NumPy arrays to PyTorch tensors
    original_tensor = torch.from_numpy(original_rgb).permute(2, 0, 1).float()  # Shape: (C, H, W)
    jpeg_compressed_85_tensor = torch.from_numpy(jpeg_compressed_85_rgb).permute(2, 0, 1).float()
    jpeg_compressed_5_tensor = torch.from_numpy(jpeg_compressed_5_rgb).permute(2, 0, 1).float()
    gaussian_noise_85_tensor = torch.from_numpy(gaussian_noise_85_rgb).permute(2, 0, 1).float()
    gaussian_noise_25_tensor = torch.from_numpy(gaussian_noise_25_rgb).permute(2, 0, 1).float()

    lpips_model = lpips.LPIPS(net='vgg')

    # Calculate LPIPS
    lpips_jpeg_85 = lpips_model(original_tensor.unsqueeze(0), jpeg_compressed_85_tensor.unsqueeze(0)).item()  # Add batch dimension
    lpips_jpeg_5 = lpips_model(original_tensor.unsqueeze(0), jpeg_compressed_5_tensor.unsqueeze(0)).item()
    lpips_noise_85 = lpips_model(original_tensor.unsqueeze(0), gaussian_noise_85_tensor.unsqueeze(0)).item()
    lpips_noise_25 = lpips_model(original_tensor.unsqueeze(0), gaussian_noise_25_tensor.unsqueeze(0)).item()

    print(f"LPIPS for JPEG compressed 85 image: {lpips_jpeg_85}")
    print(f"LPIPS for JPEG compressed 5 image: {lpips_jpeg_5}")
    print(f"LPIPS for image with Gaussian noise 85: {lpips_noise_85}")
    print(f"LPIPS for image with Gaussian noise 25: {lpips_noise_25}\n")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/Users/denisfedosov/Library/Caches/pypoetry/virtualenvs/df-p3ghjSHb-py3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/denisfedosov/Library/Caches/pypoetry/virtualenvs/df-p3ghjSHb-py3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /Users/denisfedosov/Library/Caches/pypoetry/virtualenvs/df-p3ghjSHb-py3.12/lib/python3.12/site-packages/lpips/weights/v0.1/vgg.pth


/Users/denisfedosov/Library/Caches/pypoetry/virtualenvs/df-p3ghjSHb-py3.12/lib/python3.12/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sel

LPIPS for JPEG compressed 85 image: 0.06532569229602814
LPIPS for JPEG compressed 5 image: 0.6539527773857117
LPIPS for image with Gaussian noise 85: 0.5873231291770935
LPIPS for image with Gaussian noise 25: 0.5873231291770935

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /Users/denisfedosov/Library/Caches/pypoetry/virtualenvs/df-p3ghjSHb-py3.12/lib/python3.12/site-packages/lpips/weights/v0.1/vgg.pth
LPIPS for JPEG compressed 85 image: 0.054089441895484924
LPIPS for JPEG compressed 5 image: 0.5784404873847961
LPIPS for image with Gaussian noise 85: 0.6334884166717529
LPIPS for image with Gaussian noise 25: 0.6334884166717529

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /Users/denisfedosov/Library/Caches/pypoetry/virtualenvs/df-p3ghjSHb-py3.12/lib/python3.12/site-packages/lpips/weights/v0.1/vgg.pth
LPIPS for JPEG compressed 85 image: 0.03976515308022499
LPIPS for JPEG compressed 5 image: 0.3610

libpng warning: iCCP: profile 'ICC Profile': 'RGB ': RGB color space not permitted on grayscale PNG


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /Users/denisfedosov/Library/Caches/pypoetry/virtualenvs/df-p3ghjSHb-py3.12/lib/python3.12/site-packages/lpips/weights/v0.1/vgg.pth
LPIPS for JPEG compressed 85 image: 0.012160254642367363
LPIPS for JPEG compressed 5 image: 0.130621999502182
LPIPS for image with Gaussian noise 85: 0.7949854731559753
LPIPS for image with Gaussian noise 25: 0.7949854731559753

